<a href="https://colab.research.google.com/github/K4to-0/Routput/blob/master/v1_3_pixivpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **pixiv 人気順検索もどき(プレビュー)**




### セル(各プログラムのこと)の実行方法など
https://pyhoo.jp/google-colaboratory-manual#:~:text=%E3%81%8C%E3%81%A7%E3%81%8D%E3%81%BE%E3%81%99%E3%80%82-,Google%20Colab%E3%81%AE%E3%82%BB%E3%83%AB%E3%81%AE%E5%AE%9F%E8%A1%8C%E6%96%B9%E6%B3%95%EF%BC%88shift%2Benter%EF%BC%89,%E3%81%99%E3%82%8B%E3%81%93%E3%81%A8%E3%82%82%E5%8F%AF%E8%83%BD%E3%81%A7%E3%81%99%E3%80%82

#### まずはこれを脳死で実行しよう

In [ ]:
!pip install pixivpy
!rm -rf sample_data

#### これも脳死で実行しよう

In [ ]:
#--------さわるな危険--------

from pixivpy3 import PixivAPI
from pixivpy3 import AppPixivAPI
import json
import os
from time import sleep
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
import math

searchtag = ""
eromode   = "off"

def login(id, password):
  api = PixivAPI()
  api.login(id, password)
  return api

def binary_search(apilogin,range_num):
  api = apilogin
  aapi = AppPixivAPI()

  head = 1
  tail = range_num
  while head <= tail:
    errorcount = 0
    target = (head + tail) // 2
    for i in range(2):
      json_result = api.search_works(searchtag, page=target+i, mode='tag')
      try:
        illust_len = len(json_result.response)
      except TypeError as e:
        errorcount = errorcount + 1
      sleep(0.2)
    if errorcount == 1:
      return target
    elif errorcount == 0:
      head = target + 1
    else:
      tail = target - 1

def clearLabel(_ax):
  _ax.tick_params(labelbottom="off",bottom="off")
  _ax.tick_params(labelleft="off",left="off")
  _ax.set_xticklabels([]) 
  _ax.axis('off')
  return _ax

def binary_check(apilogin,range_num):
  api = apilogin
  aapi = AppPixivAPI()

  json_result = api.search_works(searchtag, page=range_num, mode='tag')
  try:
    illust_len = len(json_result.response)
  except TypeError as e:
    binary_num = binary_search(apilogin,range_num)
    if binary_num < range_num:
      range_num = binary_num
  return range_num

def search_and_save(apilogin, searchtag, min_score, range_num, dirname,eromode):
  api = apilogin
  aapi = AppPixivAPI()
  saving_dir_path = os.path.join("/content/", dirname)
  if not os.path.exists(saving_dir_path):
    os.mkdir(saving_dir_path)

  illsut_hitsum = 0
  illsut_sum = 0
  for page in range(1, range_num + 1):
    json_result = api.search_works(searchtag, page=page, mode='tag')
    illust_len = len(json_result.response)

    illsut_sum = illsut_sum + illust_len
    illust_hit = 0

    progress = int(page / range_num * 20)
    progressbar = "["
    for i in range(progress):
      progressbar = progressbar + "*"
    for i in range(20-progress):
      progressbar = progressbar + "-"
    progressbar = progressbar + "]"
    print('\r' + "page: " + str(page) + " / " + str(range_num) + " " + str(progressbar), end = '')

    for i in range(0, illust_len):
      illust = json_result.response[i]
      score = illust.stats.score

      if score <= min_score:
        continue
      else:
        if eromode != "all":
          flag = 0
          for j in range(0,len(illust.tags)):
            if illust.tags[j] == "R-18":
              flag = 1
          if flag == 1 and eromode == "off":
            continue 
          if flag == 0 and eromode == "on":
            continue
          aapi.download(illust.image_urls.px_480mw, path=saving_dir_path)
          sleep(1)
        illust_hit = illust_hit + 1
        aapi.download(illust.image_urls.px_480mw, path=saving_dir_path)
        sleep(1)
    illsut_hitsum = illsut_hitsum + illust_hit
  print('\n' + "完了! " + str(illsut_hitsum) + " / " + str(illsut_sum) + "件を取得しました！")

def gifdelete(dirname):
  path = "/content/" + dirname

  files = os.listdir(path)
  files_file = [f for f in files if os.path.isfile(os.path.join(path, f))]

  kakuchoushi = [" "] * (len(files_file))

  for i in range(len(files_file)):
    kakuchoushi[i] = files_file[i][-4:]
    if kakuchoushi[i] == ".gif":
      os.remove(files_file[i])

def preview(dirname):
  dirname = insertbar(dirname)
  gifdelete(dirname)
  path = "/content/" + dirname

  files = os.listdir(path)
  files_file = [f for f in files if os.path.isfile(os.path.join(path, f))]

  illust_num = [" "] * (len(files_file))
  illust_name = [" "] * (len(files_file))
  for i in range(len(files_file)):
    illust_num[i]  = ""
    illust_name[i] = files_file[i][:-4]
    
    for j in range(len(files_file[i])):
      if files_file[i][j] == "_":
        break
      else:
        illust_num[i] = illust_num[i] + files_file[i][j]
  #print(illust_num)

  for i in range(10000):
    imgurl = ""
    fig = plt.figure(dpi=350)
    if 5*i > len(files_file)-1:
      break
    for j in range(5):
      imagenow = j +5*i
      if imagenow > len(files_file)-1:
        continue
      imgurl = imgurl + "    " + "https://www.pixiv.net/artworks/" + illust_num[imagenow]
      imgpath = dirname + "/" + files_file[imagenow]
      img = cv2.imread(imgpath)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      height = img.shape[0]
      width = img.shape[1]
      clearLabel(fig.add_subplot(1,5,j+1))
      plt.imshow(img)
    print(imgurl)
    plt.show()  

def insertbar(dirname):
  listname = list(dirname)
  for i in range(len(listname)):
    if listname[i] == " ":
      listname[i] = '_'
  dirname = ""
  dirname = "".join(listname)
  return dirname

def generate(searchtag,eromode):
  range_num = 666
  dirname = searchtag
  if eromode != "off":
    if eromode == "on":
      dirname = dirname + "(R)"
    else:
      dirname = dirname + "(A)"

  dirname = insertbar(dirname)
  apilogin  = login("user_awgj8237", "pixiv12345")
  binary_num = binary_check(apilogin,range_num)
  min_score = max(int((math.log(binary_num,10)**4)*700), 2000)
  print(searchtag + " タグの画像を探索しています…(1ページにつき30枚、最大666ページ)")
  search_and_save(apilogin, searchtag, min_score, binary_num, dirname, eromode)
  print()
  preview(dirname)

## **生成+プレビュー！**

In [ ]:
searchtag = "カフェ 風景"  #検索タグを入力。半角スペースで分けることで複数タグ検索可能
eromode   = "off"   #"off"=全年齢のみ "all"=両方 "on"=R-18のみ

generate(searchtag,eromode)

#### 生成したフォルダリストの確認

In [ ]:
print("生成したフォルダリスト")
print("----------------------------------------------------")
!ls

## プレビューのみしたいとき

In [ ]:
pre_dirname = "夜 風景" #ここに見たいタグ名を入力(既に生成したもの限定)
preview(pre_dirname)

#### **おまけ** 下のコマンドを実行するとダウンロードできます！　最高だね

In [ ]:
f_name = "夜 風景" # ダウンロードしたいタグ名を入力(既に生成したもの限定)

# もともとあったzipファイルを削除
!rm download.zip

# ダウンロードしたいフォルダを zip 圧縮する
f_name = insertbar(f_name)
!zip -r /content/download.zip /content/$f_name

# 圧縮した zip ファイルをダウンロードする
from google.colab import files
files.download("/content/download.zip")